Standard Imports

In [41]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from sklearn.preprocessing import scale, normalize
from scipy.stats import zscore #computes changes in terms of standard deviations
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import statsmodels.api as sm #sm.GLS(y,x) or sm.OLS(y,x)
import copy
from statsmodels.tsa.arima_model import ARMA
from scipy import stats

Import Data

In [42]:
data = pd.read_csv('r_ES50_d_cleaned_realized.csv', index_col='Date',  parse_dates=True)

In [43]:
#compute 1/N portfolio
temp = data['1/N']
data.drop(columns=['1/N'], inplace=True)
w_eqw = [1/data.shape[1] for i in range(data.shape[1])]
data['1/N'] = data.dot(w_eqw)

In [44]:
#Compute Returns from closing prices
#sap_eod_data['pct_change'] = sap_eod_data['Close'].pct_change() #computes percentage change, which is different to log return

#sap_eod_data['log_return'] = np.log(sap_eod_data['Close']) - np.log(sap_eod_data['Close'].shift(1)) # log return

#sap_eod_data['ln_return'] = np.log(1+sap_eod_data['Close'].pct_change()) # log return

In [45]:
df_mean_std = data.agg(['mean', 'std']).T

sharp_ratio = df_mean_std['mean'] / df_mean_std['std'] #The higher the better the relationship between return and risk

In [46]:
data.describe()

,ADS.DE,AI.PA,ALV.DE,ASML.AS,CS.PA,BBVA.MC,SAN.MC,BAS.DE,BAYN.DE,BMW.DE,...,SU.PA,SIE.DE,TEF.MC,FP.PA,URW.AS,UNA.AS,DG.PA,VIV.PA,VOW.DE,1/N
count,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,...,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000,5285.000000
mean,0.000601,0.000496,0.000052,0.000474,0.000260,-0.000094,-0.000050,0.000359,0.000201,0.000291,...,0.000370,0.000102,-0.000143,0.000323,0.000249,0.000390,0.000684,-0.000048,0.000252,0.000217
std,0.019164,0.015732,0.021632,0.028144,0.027866,0.021493,0.022108,0.018277,0.020366,0.020759,...,0.021010,0.023170,0.018501,0.017266,0.018114,0.014929,0.018943,0.021851,0.026014,0.013779
min,-0.166886,-0.118337,-0.166382,-0.233179,-0.203500,-0.176490,-0.221725,-0.146707,-0.194076,-0.141391,...,-0.228129,-0.428491,-0.175791,-0.181622,-0.243530,-0.107246,-0.187227,-0.294679,-0.256950,-0.145754
25%,-0.008512,-0.007434,-0.008681,-0.012210,-0.008052,-0.010500,-0.010468,-0.008615,-0.009396,-0.009818,...,-0.009874,-0.009588,-0.008579,-0.008325,-0.007711,-0.007052,-0.008318,-0.009014,-0.010261,-0.006017
50%,0.000000,0.000568,0.000353,0.000444,0.000000,0.000000,0.000000,0.000435,0.000000,0.000000,...,0.000468,0.000000,0.000000,0.000578,0.000271,0.000224,0.000135,0.000000,0.000000,0.000761
75%,0.009572,0.008415,0.009202,0.013080,0.008712,0.010152,0.010327,0.009386,0.010247,0.010228,...,0.010819,0.009700,0.008453,0.009003,0.008584,0.007798,0.009277,0.009075,0.011238,0.006722
max,0.162592,0.112783,0.233053,0.210160,0.192927,0.199073,0.208774,0.176407,0.330062,0.138356,...,0.151643,0.435903,0.164084,0.139762,0.167360,0.116166,0.182523,0.202723,0.902682,0.104980


PCA Analysis

In [47]:
pca = PCA()

pca_comp = pca.fit_transform(zscore(data))

pca.explained_variance_ratio_

pcas = pd.DataFrame(data=pca_comp, index=data.index)

pc1 = pcas.iloc[:,0]

Standardize using zscore

In [48]:
#zscore(data)

#scale(data)

data.apply(zscore)

,ADS.DE,AI.PA,ALV.DE,ASML.AS,CS.PA,BBVA.MC,SAN.MC,BAS.DE,BAYN.DE,BMW.DE,...,SU.PA,SIE.DE,TEF.MC,FP.PA,URW.AS,UNA.AS,DG.PA,VIV.PA,VOW.DE,1/N
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-05,-1.283621,-0.624764,-0.221352,-3.578558,-0.561518,-1.052253,-1.110458,-2.150297,-1.331139,-0.976907,...,-2.059845,-1.490252,-2.578986,-1.494511,-0.065909,1.415768,1.150000,-1.114201,-1.468482,-1.672044
2000-01-06,2.158616,4.380705,1.861432,-2.593574,-0.288654,0.296547,0.002275,0.185816,0.157240,-0.170552,...,2.335955,-0.535541,0.054968,-1.725161,1.618312,2.398010,1.695011,-0.537793,-0.179354,0.426928
2000-01-07,-0.751185,-0.814621,1.265788,2.541890,1.095060,0.294724,1.962266,2.179846,1.995106,-0.101209,...,0.843482,1.698018,0.054926,0.269215,-0.356731,0.936059,1.640068,0.372395,1.083602,1.713298
2000-01-10,0.752760,-2.285201,0.263419,2.454048,-0.009346,-0.750695,-0.399052,0.220590,1.741706,1.868841,...,-0.816643,2.490551,1.613215,0.029180,0.581891,-0.796931,-2.282930,3.939767,0.097947,0.685639
2000-01-11,-0.519168,-0.617456,-0.425752,0.417526,-0.834444,-0.939194,-1.556388,-2.504947,-1.484559,-0.182163,...,-1.882159,0.292244,0.053619,0.410175,0.237769,-0.866780,2.210665,0.852950,0.587570,-0.352062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-25,0.475416,0.295024,0.703729,0.010745,0.305117,0.599061,1.121855,0.035444,-1.476856,0.751685,...,0.259277,0.360649,1.254581,0.443761,1.185565,-0.122268,0.559394,0.380724,0.720039,0.568956
2020-06-26,-1.139409,-0.408532,-0.713767,0.203180,-0.094485,-1.115047,-1.404249,-1.131934,-0.840936,-1.132575,...,0.345823,-0.122901,-1.084888,-0.125721,-2.928218,0.412141,-0.488637,-0.136876,-0.359301,-0.574869
2020-06-29,-0.390967,0.670328,0.570608,-0.214824,0.172299,1.290154,1.789587,1.569543,0.070944,0.791260,...,0.614191,0.776829,1.351313,0.123982,0.630159,-1.740727,0.877797,0.398491,0.393441,0.900110


Annualize returns and volatility

In [49]:
#returns * 252 #annualize returns

#vol * np.sqrt(252) #annualize vol

GLS/OLS Regression

In [50]:
data['rf'] = 0.1

data_es50 = data.drop(columns=['1/N', 'rf'], axis=1)

In [51]:
data['ADS.DE'] - data['rf']

Date
2000-01-05   -0.123996
2000-01-06   -0.058036
2000-01-07   -0.113793
2000-01-10   -0.084975
2000-01-11   -0.109347
                ...   
2020-06-25   -0.090289
2020-06-26   -0.121232
2020-06-29   -0.106891
2020-06-30   -0.090538
2020-07-01   -0.105150
Length: 5285, dtype: float64

In [52]:
x = data['1/N'] - data['rf']
X = sm.add_constant(x)

y = data['ADS.DE'] - data['rf']

result = sm.OLS(y,X).fit()

result.summary() #46% explained by 7 Factors

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     2414.
Date:                Sat, 30 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:18:05   Log-Likelihood:                 14397.
No. Observations:                5285   AIC:                        -2.879e+04
Df Residuals:                    5283   BIC:                        -2.878e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0217      0.002    -13.577      0.000      -0.025      -0.019
0              0.7789      0.016     49.134      0.000       0.748       0.810
==============================================================================
Omnibus:                      971.396   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22372.069
Skew:                           0.211   Prob(JB):                         0.00
Kurtosis:                      13.071   Cond. No.                         73.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
result.pvalues.values[1:]

array([0.])

AutoRegression and Vectorautoregression

In [54]:
#always use zscored input to make it stationary and have an impulse shock of 1 standard deviation
model = VAR(data_es50.apply(zscore))

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [55]:
x = model.select_order(maxlags=12)
x.summary() #choose values lag length with star

,AIC,BIC,FPE,HQIC
0,-27.72,-27.67*,9.122e-13,-27.70*
1,-28.09,-25.74,6.288e-13,-27.27
2,-28.20,-23.54,5.637e-13,-26.58
3,-28.21*,-21.25,5.604e-13*,-25.78
4,-28.19,-18.93,5.697e-13,-24.95
5,-28.19,-16.62,5.712e-13,-24.15
6,-28.16,-14.28,5.912e-13,-23.31
7,-28.09,-11.91,6.376e-13,-22.43
8,-28.04,-9.559,6.667e-13,-21.58
9,-27.99,-7.202,7.057e-13,-20.72


In [56]:
model_fitted = model.fit(1) # VAR(1)
#model_fitted.summary()

Impulse Response Function

In [57]:

#periods = 5

#model_fitted.irf(periods).plot(impulse='0.25 Y', response='1/N', orth=True, stderr_type="asym", signif=0.05) #normalerweise ortho=True, außer bei irf zwischen principal component und ursprünglichen vektor

Granger Causality Check

In [59]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

#granger_result = grangers_causation_matrix(df_1n_ecb, variables= df_1n_ecb.columns) #low p-value mean relationship

ARMA(p,q)

In [60]:
#check for predictability
'''returns: return array i.e. returns.values'''
def selectARMAorder(max_p,max_q,returns):
    bic_vals = np.zeros((max_p+1,max_q+1))
    for i in range(max_p+1):
        for j in range(max_p+1):
            arma_model = ARMA(returns,(i,j))
            bic_vals[i,j] = arma_model.fit().bic
    opt = np.argmin(bic_vals)
    opt_model = np.unravel_index(opt, bic_vals.shape)
    return opt_model


In [63]:
opt_arma_order = selectARMAorder(2,2,data['1/N'])
print(opt_arma_order) # (0,0) indicates constant mean fits best according to bic
arma_model = ARMA(data['1/N'],opt_arma_order)
arma_result = arma_model.fit()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency

(0, 0)


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency

R_Squared as Variance Measure

Stochastic Volatility Models: Harvey & Garch

In [64]:
#Garch
from arch import arch_model 

##GARCH(1,1) 
equityVol =  arch_model(zscore(data['1/N']), p=1, q=1, mean='zero', vol = 'GARCH', dist = 'normal')  
equityVol_result = equityVol.fit(update_freq=4)


data["data_equity_garch_vol"] =  equityVol_result.conditional_volatility

Iteration:      4,   Func. Count:     22,   Neg. LLF: 6441.6609189781375
Iteration:      8,   Func. Count:     42,   Neg. LLF: 6428.952368679707
Iteration:     12,   Func. Count:     57,   Neg. LLF: 6428.950183003318
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6428.950183003191
            Iterations: 12
            Function evaluations: 57
            Gradient evaluations: 12


In [65]:
#Harvey
# https://mc-stan.org/docs/2_21/stan-users-guide/stochastic-volatility-models.html
%run Helper_SVM_Harvey.ipynb

In [66]:
svm = QLSV(data['1/N'] - data['1/N'].mean()) #Either you use zscore for the vol model or demeaned returns; In case of zscore, the volatility is in terms of standard deviations
svm_result = svm.fit(cov_type='robust')

data["data_equity_harvey_cond_vol"] = np.exp(svm_result.smoothed_state[0] / 2) #nur smoothed_state[0] ist die log volatilty

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [67]:
print(svm_result.summary())

                           Statespace Model Results                           
Dep. Variable:                    1/N   No. Observations:                 5285
Model:                           QLSV   Log Likelihood              -11890.056
Date:                Sat, 30 Jan 2021   AIC                          23786.112
Time:                        18:21:48   BIC                          23805.830
Sample:                             0   HQIC                         23793.004
                               - 5285                                         
Covariance Type:               robust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
phi            0.9863      0.003    300.005      0.000       0.980       0.993
sigma2_eta     0.0256      0.005      4.677      0.000       0.015       0.036
mu            -9.1449      0.163    -56.177      0.0

Fama/MacBeth Model

In [68]:
%run Helper_FamaMacBeth.ipynb

In [69]:
import copy

class FMacBethAlex():
    def __init__(self, factor_returns, returns, risk_free_rate):
        #input: returns: return panel, pandas
        #input: factor_returns: factor panel, pandas
        #input: rf: risk-free rate, pandas series
        
        self.r = copy.copy(returns)
        self.f = copy.copy(factor_returns)
        self.rf = copy.copy(risk_free_rate)
        
        self.beta_panel = pd.DataFrame()
        
        self.lambda_panel = pd.DataFrame()
        
        self.r_squared_beta_panel = {}
        
        self.beta_pvalues = {}
        
        self.lambda_pvalues = {}
        
        self.r_squared_lambda_panel = {}
        
        self.df_betas_internal = {}
        
        
    def compute_betas(self):
    
        df_betas = pd.DataFrame()
        temp_beta_array = []
        
        adj_r = []
        
        pvalues = []
        
        for stock in self.r.columns:
            
        
            x = self.f
            X = sm.add_constant(x)
            y = self.r[stock] - self.rf
            result = sm.GLS(y,X).fit()
            
            #save beta/coefficient for each stock
            temp_beta_array.append(result.params.values[1:])
            #print(stock,': ', result.rsquared_adj)
            adj_r.append(result.rsquared_adj)
            
            pvalues.append(result.pvalues.values[1:])
            
            #print('Max: ' ,max(adj_r))
            
            #df_betas[factor] = temp_beta_array
        
        
        self.r_squared_beta_panel = adj_r
        df_betas = pd.DataFrame(data = temp_beta_array , columns=self.f.columns)
        df_betas_pvalues = pd.DataFrame(data = pvalues , columns=self.f.columns)
        #set stock names as index
        df_betas.set_index(returns.columns, inplace=True)
    
        self.df_betas_internal = df_betas.copy()
    
        df_betas['r_squared'] = adj_r
        
    
        #return betas
        self.beta_pvalues = df_betas_pvalues.copy()
        self.beta_panel = df_betas.copy()
        
    def cross_regress(self):
    
        #betas are x
        x = self.df_betas_internal
        X = sm.add_constant(x)
        
        temp_coeff_array = []
        i = 0
        
        lambda_adj_r = []
        pvalues = []
        
        for row in self.r.to_numpy():
            y = row - self.rf.iloc[i]
            result = sm.GLS(y, X).fit()
            temp_coeff_array.append(result.params.values[1:])
            i = i + 1
            lambda_adj_r.append(result.rsquared_adj)
            pvalues.append(result.pvalues.values[1:])
        
        self.lambda_pvalues = pd.DataFrame(data = pvalues, columns = self.df_betas_internal.columns, index=self.r.index)
        
        market_premiums = pd.DataFrame(data = temp_coeff_array, columns = self.df_betas_internal.columns, index=self.r.index)
        
        market_premiums['r_squared']= lambda_adj_r
        
        self.lambda_panel =  market_premiums.copy()
        

def compute_betas(factor_returns, returns, risk_free_rate):
    
    df_betas = pd.DataFrame()
    temp_beta_array = []
    
    adj_r = []
    
    for stock in returns.columns:
        
    
        x = factor_returns
        X = sm.add_constant(x)
        y = returns[stock] - risk_free_rate
        result = sm.GLS(y,X).fit()
        
        #save beta/coefficient for each stock
        temp_beta_array.append(result.params.values[1:])
        print(stock,': ',result.rsquared_adj)
        
        adj_r.append(stock +': ' + str(result.rsquared_adj))
        
        #print('Max: ' ,max(adj_r))
        
        #df_betas[factor] = temp_beta_array
    
    df_betas = pd.DataFrame(data = temp_beta_array, columns=factor_returns.columns)
    #set stock names as index
    df_betas.set_index(self.r.columns, inplace=True)
    
    #return betas
    return df_betas 

def cross_regress(df_betas, returns, risk_free_rate):
    
    #betas are x
    x = df_betas
    X = sm.add_constant(x)
    
    temp_coeff_array = []
    i = 0
    
    for row in returns.to_numpy():
        y = row - risk_free_rate.iloc[i]
        result = sm.GLS(y, X).fit()
        temp_coeff_array.append(result.params.values[1:])
        i = i + 1
        
    
    market_premiums = pd.DataFrame(data = temp_coeff_array, columns = df_betas.columns)
    
    #market_premiums = market_premiums.agg(func=['mean'])
    
    return market_premiums

In [70]:
returns = data.iloc[:, :43]

market_factor = data['1/N'] # just the market factor here i.e. CAPM

rf = pd.DataFrame(data=data['rf'], index=data.index) #is zero here

In [71]:
fMacBeth = FMacBeth(returns, market_factor.to_frame(), rf) # (return panel, factor panel, risk-free panel )

fMacBeth.TSRegression_step1()

fMacBeth.CSRegression_step2()

ValueError: could not broadcast input array from shape (7) into shape (1)

In [72]:
fmacAlex = FMacBethAlex(market_factor.to_frame(), returns, rf['rf'])

fmacAlex.compute_betas()

fmacAlex.cross_regress()

fmacAlex.beta_panel

,1/N,r_squared
ADS.DE,0.778880,0.313515
AI.PA,0.830393,0.528900
ALV.DE,1.253785,0.637748
ASML.AS,1.213499,0.352879
CS.PA,1.524956,0.568542
BBVA.MC,1.264213,0.656809
SAN.MC,1.299451,0.655874
BAS.DE,1.022419,0.594059
BAYN.DE,0.980623,0.440073
BMW.DE,1.068446,0.502907


In [73]:
fmacAlex.beta_pvalues

,1/N
0,0.000000e+00
1,0.000000e+00
2,0.000000e+00
3,0.000000e+00
4,0.000000e+00
5,0.000000e+00
6,0.000000e+00
7,0.000000e+00
8,0.000000e+00
9,0.000000e+00


In [74]:
fmacAlex.lambda_panel

,1/N,r_squared
Date,,
2000-01-05,-0.024198,0.007292
2000-01-06,-0.065702,0.210345
2000-01-07,0.023771,0.006560
2000-01-10,0.026046,0.032383
2000-01-11,-0.025133,0.042967
...,...,...
2020-06-25,0.010508,0.031833
2020-06-26,-0.010988,0.010605
2020-06-29,0.026928,0.213016


In [75]:
fmacAlex.lambda_pvalues

,1/N
Date,
2000-01-05,0.259301
2000-01-06,0.001166
2000-01-07,0.264965
2000-01-10,0.128586
2000-01-11,0.096950
...,...
2020-06-25,0.130508
2020-06-26,0.235407
2020-06-29,0.001082


Options

In [76]:
from black_scholes import IV_BS_Newton

In [78]:
sap_atm_options['bs_impl_vol'] = IV_BS_Newton(sap_atm_options["put_price"].values, sap_atm_options["futures_price"].values,\
                sap_atm_options["exercise_price"].values, sap_atm_options["dtm"].values/360,\
                sap_atm_options["rf"].values,type_='P', underlying='future')

IndentationError: unexpected indent (<ipython-input-78-c271bdb8fd97>, line 2)

Kalman Filter

Stationarity Test / Adfuller

In [79]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='BIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [80]:
for name, column in data.iteritems():
    #adfuller_test(column, name=column.name)
    #print('\n')

SyntaxError: unexpected EOF while parsing (<ipython-input-80-0336fdba8f60>, line 3)

In [81]:
#Make Stationary by differentiating:
data_stat = data.diff().dropna()

In [82]:
#Make Stationary by zscore (Changes distribution):
zscore(data)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


array([[-1.28362079, -0.62476374, -0.22135174, ...,         nan,
         0.13229796,  0.22354491],
       [ 2.15861576,  4.38070495,  1.86143156, ...,         nan,
         0.35621782,  0.2154514 ],
       [-0.75118482, -0.81462096,  1.26578844, ...,         nan,
         0.24853634,  0.20891037],
       ...,
       [-0.39096742,  0.67032781,  0.57060822, ...,         nan,
         1.02733352,  0.97213746],
       [ 0.46245311,  0.41559261,  0.3909521 , ...,         nan,
         0.92761391,  0.93581899],
       [-0.30014201, -0.45378276, -0.89097026, ...,         nan,
         0.76246363,  0.92301426]])

Normalverteilung/Jarque-Berra Test

In [83]:
stats.jarque_bera(data["ADS.DE"]) #Null Hypothese nimmt Normalverteilung an. pvalue=0.1: 10% Wahrscheinlichkeit Nullhypothese ist korrekt.

Jarque_beraResult(statistic=12784.531958116217, pvalue=0.0)

Systematic Risk / Unsystematic Risk

In [84]:

'''
data_std_mean = data.agg(['mean', 'std']).T
data_std_mean['corrWith1/N'] = data.corr()['1/N'] 
data_std_mean['Total.Risk'] = (data_std_mean['std'] ** 2) * 252

data_std_mean['System.Risk'] = data_std_mean['std']['1/N'] * data_std_mean['std'] * data_std_mean['corrWith1/N']

data_std_mean['System.Risk'] = data_std_mean['System.Risk'] * 252
data_std_mean['System.Risk.Ratio'] =  data_std_mean['System.Risk'] / data_std_mean['Total.Risk']

'''

"\ndata_std_mean = data.agg(['mean', 'std']).T\ndata_std_mean['corrWith1/N'] = data.corr()['1/N'] \ndata_std_mean['Total.Risk'] = (data_std_mean['std'] ** 2) * 252\n\ndata_std_mean['System.Risk'] = data_std_mean['std']['1/N'] * data_std_mean['std'] * data_std_mean['corrWith1/N']\n\ndata_std_mean['System.Risk'] = data_std_mean['System.Risk'] * 252\ndata_std_mean['System.Risk.Ratio'] =  data_std_mean['System.Risk'] / data_std_mean['Total.Risk']\n\n"

In [85]:
X = sm.add_constant(data['1/N'].copy())

gls = sm.GLS(data['AI.PA'], X)

result = gls.fit()

result.summary() #r_squared is systematic risk, 1/N coefficient is beta

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                  AI.PA   R-squared:                       0.529
Model:                            GLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                     5933.
Date:                Sat, 30 Jan 2021   Prob (F-statistic):               0.00
Time:                        18:23:56   Log-Likelihood:                 16434.
No. Observations:                5285   AIC:                        -3.286e+04
Df Residuals:                    5283   BIC:                        -3.285e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.000      2.130      0.033    2.51e-05       0.001
1/N            0.8304      0.011     77.028      0.000       0.809       0.852
==============================================================================
Omnibus:                     1106.368   Durbin-Watson:                   2.215
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18619.991
Skew:                           0.537   Prob(JB):                         0.00
Kurtosis:                      12.133   Cond. No.                         72.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Correlation Analysis

In [86]:
np.corrcoef(data['AI.PA'], data['1/N'])

array([[1.        , 0.72731644],
       [0.72731644, 1.        ]])

In [87]:
data.corr().head(5)

,ADS.DE,AI.PA,ALV.DE,ASML.AS,CS.PA,BBVA.MC,SAN.MC,BAS.DE,BAYN.DE,BMW.DE,...,FP.PA,URW.AS,UNA.AS,DG.PA,VIV.PA,VOW.DE,1/N,rf,data_equity_garch_vol,data_equity_harvey_cond_vol
ADS.DE,1.000000,0.388807,0.450709,0.304632,0.394184,0.387748,0.389301,0.480341,0.380800,0.445458,...,0.386752,0.327038,0.316999,0.391919,0.295164,0.252050,0.560040,NaN,-0.004194,-0.038822
AI.PA,0.388807,1.000000,0.552326,0.402729,0.536640,0.550599,0.558219,0.620586,0.524982,0.504605,...,0.563838,0.408094,0.427068,0.497231,0.438899,0.310669,0.727316,NaN,0.015338,-0.029883
ALV.DE,0.450709,0.552326,1.000000,0.426367,0.666409,0.638084,0.643762,0.658900,0.539662,0.572547,...,0.564897,0.423604,0.376973,0.517459,0.481930,0.290527,0.798634,NaN,-0.015398,-0.073172
ASML.AS,0.304632,0.402729,0.426367,1.000000,0.406535,0.431757,0.430780,0.407899,0.369855,0.366333,...,0.350195,0.241542,0.211998,0.302714,0.437071,0.272956,0.594140,NaN,0.017989,-0.028026
CS.PA,0.394184,0.536640,0.666409,0.406535,1.000000,0.638593,0.636118,0.551941,0.438445,0.518278,...,0.530296,0.405684,0.348002,0.484518,0.475218,0.308257,0.754072,NaN,-0.006264,-0.045597
